## Importing needed libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

## Loading data

In [20]:
df = pd.read_csv('./data/movies.csv')
df.head()

## Creating a movie recommendation system

### Data preparation

Assigning grade ranges

In [21]:
df_copy = df.copy()
df_copy.loc[(df_copy['vote_average'] >= 0) & (df_copy['vote_average'] <= 1), 'rating_between'] = "between 0 and 1"
df_copy.loc[(df_copy['vote_average'] > 1) & (df_copy['vote_average'] <= 2), 'rating_between'] = "between 1 and 2"
df_copy.loc[(df_copy['vote_average'] > 2) & (df_copy['vote_average'] <= 3), 'rating_between'] = "between 2 and 3"
df_copy.loc[(df_copy['vote_average'] > 3) & (df_copy['vote_average'] <= 4), 'rating_between'] = "between 3 and 4"
df_copy.loc[(df_copy['vote_average'] > 4) & (df_copy['vote_average'] <= 5), 'rating_between'] = "between 4 and 5"
df_copy.loc[(df_copy['vote_average'] > 5) & (df_copy['vote_average'] <= 6), 'rating_between'] = "between 5 and 6"
df_copy.loc[(df_copy['vote_average'] > 6) & (df_copy['vote_average'] <= 7), 'rating_between'] = "between 6 and 7"
df_copy.loc[(df_copy['vote_average'] > 7) & (df_copy['vote_average'] <= 8), 'rating_between'] = "between 7 and 8"
df_copy.loc[(df_copy['vote_average'] > 8) & (df_copy['vote_average'] <= 9), 'rating_between'] = "between 8 and 9"
df_copy.loc[(df_copy['vote_average'] > 9) & (df_copy['vote_average'] <= 10), 'rating_between'] = "between 9 and 10"

Filling in missing values

In [22]:
df_copy['genres'] = df_copy['genres'].fillna('no category')

Creating categorical variables

In [23]:
rating_df = pd.get_dummies(df_copy['rating_between'])
language_df = pd.get_dummies(df_copy['original_language'])

Extracting the main category

In [24]:
df_copy['main_category'] = df_copy['genres'].apply(lambda x: x.split('-')[0])
genres_df = pd.get_dummies(df_copy.main_category)

## Creating features

In [25]:
features = pd.concat([rating_df, language_df, genres_df, df_copy['vote_average'], df_copy['vote_count']], axis=1)

### Feature scaling

In [26]:
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features)

## Division into training and test set

In [27]:
X = features
y = df_copy[['title', 'main_category']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Creating a KNN model

In [28]:
n_neighbors = 10
knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(X_train)

## Recommender function

In [29]:
def recommend_movies(title, n_recommendations=5):
    if title not in y_train['title'].values:
        return "Film nie znaleziony w bazie treningowej."
    
    idx = y_train[y_train['title'] == title].index[0]
    movie_features = X_train[idx]
    
    # Zmiana kształtu na (1, -1)
    movie_features = movie_features.reshape(1, -1)
    
    distances, indices = knn.kneighbors(movie_features, n_neighbors=n_recommendations + 1)
    
    recommendations = []
    for i in range(1, len(indices[0])):
        recommendations.append(y_train.iloc[indices[0][i]])
    
    return recommendations

## Testing the model

In [30]:
recommendations = recommend_movies("Toy Story", n_recommendations=5)
for title, genres in recommendations:
    print(f"Film: {title}, Kategorie: {genres}")

In [32]:
recommendations = recommend_movies("Music", n_recommendations=30)
for title, genres in recommendations:
    print(f"Film: {title}, Kategorie: {genres}")